El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from time import time
import joblib
from sklearn.metrics import mean_squared_error
import numpy as np

In [29]:
# Cargar el dataset /datasets/
df = pd.read_csv('car_data.csv')

In [30]:
# Exploración inicial
print(df.info())
print(df.describe())
print(df.isnull().sum())
print(df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [31]:
# Eliminar duplicados
df = df.drop_duplicates()
# Eliminar filas con nulos restantes
df = df.dropna()
# Codificación para modelos simples
label_cols = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
for col in label_cols:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

# Eliminación de columnas irrelevantes
df = df.drop(['DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures', 'PostalCode'], axis=1)


se procedio a eliminar los datos duplicados y los valores nulos, para tener un dataset limpio y listo para el entrenamiento del modelo.

In [32]:
# División en características y objetivo
features = df.drop('Price', axis=1)
target = df['Price']

# Separación en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=42)


## Entrenamiento del modelo 

In [33]:
# Función de evaluación
def evaluate_model(model):
    start_train = time()
    model.fit(X_train, y_train)
    train_time = time() - start_train

    start_pred = time()
    preds = model.predict(X_test)
    pred_time = time() - start_pred

    mse = mean_squared_error(y_test, preds)
    rmse = np.sqrt(mse)

    return rmse, train_time, pred_time

# Modelos y parámetros iniciales
models = {
    'Regresión lineal': LinearRegression(),
    'Árbol de decisión': DecisionTreeRegressor(max_depth=10, min_samples_split=10),
    'Bosque aleatorio': RandomForestRegressor(n_estimators=100, max_depth=10, max_features='sqrt'),
    'LightGBM': LGBMRegressor(num_leaves=31, learning_rate=0.1, n_estimators=100)
}

# Evaluación
results = {}
for name, model in models.items():
    rmse, train_t, pred_t = evaluate_model(model)
    results[name] = {'RECM': rmse, 'Tiempo de entrenamiento': train_t, 'Tiempo de predicción': pred_t}

df_results = pd.DataFrame(results).T
print(df_results)



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006016 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 660
[LightGBM] [Info] Number of data points in the train set: 184175, number of used features: 10
[LightGBM] [Info] Start training from score 5127.092700
                          RECM  Tiempo de entrenamiento  Tiempo de predicción
Regresión lineal   3395.682779                 0.052633              0.007321
Árbol de decisión  2027.310643                 0.325611              0.009388
Bosque aleatorio   1897.870566                 8.105708              0.444203
LightGBM           1738.387391                 0.541162              0.095656


Podemos observar que el modelo de LightGBM tiene el mejor rendimiento en términos de RMSE, seguido por el Bosque Aleatorio. El Árbol de Decisión mejor que la Regresión Lineal pero no logra superar a los modelos más complejos, regresion lineal muy veloz per su error es el mas alto.

## Análisis del modelo

In [34]:
# Entrenamiento del mejor modelo
best_model = LGBMRegressor(num_leaves=31, learning_rate=0.1, n_estimators=100)
best_model.fit(X_train, y_train)

# Guardar el modelo
joblib.dump(best_model, 'modelo_precio_autos.pkl')

# Cargar el modelo guardado
modelo_cargado = joblib.load('modelo_precio_autos.pkl')

# Hacer predicciones con nuevos datos
nuevas_predicciones = modelo_cargado.predict(X_test)

rmse_final = np.sqrt(mean_squared_error(y_test, nuevas_predicciones))
print(f"RECM del modelo cargado: {rmse_final:.2f}")

df_results.to_csv('resultados_modelos.csv', index=True)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004993 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 660
[LightGBM] [Info] Number of data points in the train set: 184175, number of used features: 10
[LightGBM] [Info] Start training from score 5127.092700
RECM del modelo cargado: 1738.39


Como conclusion tenemos que el modelo de LightGBM es el que mejor rendimiento ha tenido en este caso. El error cuadrático medio (RMSE) obtenido fue de 1738.39 es bastante aceptable para un modelo de este tipo. el tiempo del entrenamiento fue de 0.004 segundo es decir es muy rápido. El modelo utiliza 10 variables predictorias esto es eficiente, y su capacidad para manejar grandes volúmenes de datos y realizar predicciones rápidas lo convierte en una excelente opción para problemas de regresión en el ámbito automotriz.
